domingo 16 de mayo de 2019  
  
_Benjamín Hernández Cortés_ - _Juan Pablo Rojas Rojas_  
_Departamento de Ingeniería Informática (DIINF)_  
_Universidad de Santiago de Chile (USACH)_


## Laboratorio 3 - Fundamentos de Aprendizaje Profundo con Redes Neuronales
___

El presente código está orientado hacia la implementación de una red neuronal convolucional o CNN (Convolutional Neural Network), la cual será diseñada y empleada con el fin de clasificar un conjunto de 25.000 imagenes de perros y gatos, que pueden ser descargados a través del siguiente enlace: [Kaggle Cats and Dogs Dataset](https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765).


#### Importación de bibliotecas
---

Las bibliotecas a emplear son:
- **os:** Manejo de funcionalidades dependientes del sistema operativo
- **numpy:** Herramienta de computación científica, que permite trabajar a través de vectores
- **matplotlib:** Visualización gráfica de diversos datos de interés
- **tensorflow:** Diseño e implementación de la red neuronal convolucional
- **cv2:** Manejo del conjunto de imagenes
- **random:** Uso de funciones relacionadas con la aleatoriedad
- **tqdm:** Barra de progreso interactiva

#### ¡Precauciones!
---

Para asegurar el correcto funcionamiento de la herramienta, es necesario tener en cuenta las siguientes precauciones y las acciones a tomar, para el reparar algunas funcionalidades que podrían presentar problemas.

| <p style='text-align: left;'>**Error**</p>  | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| <p style='text-align: justify;'>`UnboundLocalError: local variable 'photoshop' referenced before assignment`</p> |  <p style='text-align: justify;'>Este mensaje de error se presenta al momento de utilizar la clase _ImageDataGenerator_ de _tensorflow.keras.preprocessing.image_. Este error puede ser reparado haciendo una leve modificación al archivo _JpegImagePlugin.py_ asociado a la biblioteca _PIL_, siguiendo los pasos indicados en este [post](https://github.com/python-pillow/Pillow/pull/3771#issuecomment-485104596).</p>|
| <p style='text-align: justify;'>`Problema al obtener/cargar las imagenes`</p> |  <p style='text-align: justify;'>El conjunto de imagenes indicado al principio de este trabajo ([Kaggle Cats and Dogs Dataset](https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765)), posee un par de imagenes que no contienen información y por lo tanto, no pueden ser empleados. Las imagenes en cuestión son _/PetImages/Cat/666.jpg_ y _/PetImages/Dog/11702.jpg_. Se recomienda que elimine las imagenes anteriormente indicadas, previo a la ejecución de este archivo.</p>|

In [32]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

from random import shuffle
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization

In [22]:
IMAGE_SIZE = 128
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
BATCH_SIZE = 32
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)
DIR = "C:\\Users\\Familia Hernández\\Desktop\\Redes_neuronales_1-2019\\Laboratorio 3 - Redes Convolucionales\\PetImages\\Cat"


datagen = ImageDataGenerator(rescale=1.0/255.0,
                             validation_split=0.3)

# Conjunto de prueba
training_dataset = datagen.flow_from_directory('PetImages/',
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               batch_size=100,
                                               color_mode='rgb',
                                               class_mode='binary',
                                               shuffle=True,
                                               subset='training')

# Conjunto de validación
validation_dataset = datagen.flow_from_directory('PetImages/',
                                                 target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                 batch_size=100,
                                                 color_mode='grayscale',
                                                 class_mode='binary',
                                                 shuffle=True,
                                                 subset='validation')



model = Sequential()
model.add(Conv2D(32, (7, 7), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Dense(1, activation='softmax'))

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

model.summary()

model.fit_generator(training_dataset,
                    steps_per_epoch=len(training_dataset),
                    epochs = 10,
                    validation_steps = len(validation_dataset),
                    validation_data = validation_dataset);


Found 17500 images belonging to 2 classes.
Found 7498 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 250, 250, 32)      4736      
_________________________________________________________________
batch_normalization_v1_8 (Ba (None, 250, 250, 32)      128       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 125, 125, 32)      0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 123, 123, 64)      18496     
_________________________________________________________________
batch_normalization_v1_9 (Ba (None, 123, 123, 64)      256       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 61, 61, 64)        0         
_________________________________________________________

KeyboardInterrupt: 

In [20]:
len(training_dataset)

175

In [36]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir("PetImages/Cat")):
        label = [0]
        path = os.path.join("PetImages/Cat", img)
        print("Path => ", path)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128,128))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

a = create_train_data()

  0%|          | 0/12500 [00:00<?, ?it/s]

Path =>  PetImages/Cat\0.jpg
Path =>  PetImages/Cat\1.jpg
Path =>  PetImages/Cat\10.jpg
Path =>  PetImages/Cat\100.jpg
Path =>  PetImages/Cat\1000.jpg
Path =>  PetImages/Cat\10000.jpg
Path =>  PetImages/Cat\10001.jpg
Path =>  PetImages/Cat\10002.jpg
Path =>  PetImages/Cat\10003.jpg
Path =>  PetImages/Cat\10004.jpg
Path =>  PetImages/Cat\10005.jpg
Path =>  PetImages/Cat\10006.jpg
Path =>  PetImages/Cat\10007.jpg
Path =>  PetImages/Cat\10008.jpg
Path =>  PetImages/Cat\10009.jpg
Path =>  PetImages/Cat\1001.jpg
Path =>  PetImages/Cat\10010.jpg
Path =>  PetImages/Cat\10011.jpg
Path =>  PetImages/Cat\10012.jpg
Path =>  PetImages/Cat\10013.jpg


  0%|          | 20/12500 [00:00<01:06, 186.92it/s]

Path =>  PetImages/Cat\10014.jpg
Path =>  PetImages/Cat\10015.jpg
Path =>  PetImages/Cat\10016.jpg
Path =>  PetImages/Cat\10017.jpg
Path =>  PetImages/Cat\10018.jpg
Path =>  PetImages/Cat\10019.jpg
Path =>  PetImages/Cat\1002.jpg
Path =>  PetImages/Cat\10020.jpg
Path =>  PetImages/Cat\10021.jpg
Path =>  PetImages/Cat\10022.jpg
Path =>  PetImages/Cat\10023.jpg
Path =>  PetImages/Cat\10024.jpg
Path =>  PetImages/Cat\10025.jpg
Path =>  PetImages/Cat\10026.jpg
Path =>  PetImages/Cat\10027.jpg
Path =>  PetImages/Cat\10028.jpg


  0%|          | 36/12500 [00:00<01:10, 176.17it/s]

...

  1%|          | 106/12500 [00:00<01:11, 173.93it/s]

Path =>  PetImages/Cat\10092.jpg
Path =>  PetImages/Cat\10093.jpg
Path =>  PetImages/Cat\10094.jpg
Path =>  PetImages/Cat\10095.jpg
Path =>  PetImages/Cat\10096.jpg
Path =>  PetImages/Cat\10097.jpg
Path =>  PetImages/Cat\10098.jpg
Path =>  PetImages/Cat\10099.jpg
Path =>  PetImages/Cat\101.jpg
Path =>  PetImages/Cat\1010.jpg
Path =>  PetImages/Cat\10100.jpg
Path =>  PetImages/Cat\10101.jpg
Path =>  PetImages/Cat\10102.jpg
Path =>  PetImages/Cat\10103.jpg
Path =>  PetImages/Cat\10104.jpg
Path =>  PetImages/Cat\10105.jpg
Path =>  PetImages/Cat\10106.jpg
Path =>  PetImages/Cat\10107.jpg
Path =>  PetImages/Cat\10108.jpg
Path =>  PetImages/Cat\10109.jpg
Path =>  PetImages/Cat\1011.jpg


  1%|          | 127/12500 [00:00<01:08, 181.01it/s]

Path =>  PetImages/Cat\10110.jpg
Path =>  PetImages/Cat\10111.jpg
Path =>  PetImages/Cat\10112.jpg
Path =>  PetImages/Cat\10113.jpg
Path =>  PetImages/Cat\10114.jpg
Path =>  PetImages/Cat\10115.jpg
Path =>  PetImages/Cat\10116.jpg
Path =>  PetImages/Cat\10117.jpg
Path =>  PetImages/Cat\10118.jpg
Path =>  PetImages/Cat\10119.jpg
Path =>  PetImages/Cat\1012.jpg
Path =>  PetImages/Cat\10120.jpg
Path =>  PetImages/Cat\10121.jpg
Path =>  PetImages/Cat\10122.jpg
Path =>  PetImages/Cat\10123.jpg
Path =>  PetImages/Cat\10124.jpg
Path =>  PetImages/Cat\10125.jpg


error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize
